# Iterables, Iterators, and Generators

可迭代的对象，迭代器和生成器

迭代是数据处理的基石。扫描内存中放不下的数据集时，我们要找到一种惰性获取数据的方式，即按需一次获取一个数据项。这就是迭代器模式（Iterator Pattern）。

> 所有生成器都是迭代器，因为生成器完全实现了迭代器的接口。不过，根据《设计模式：可复用面向对象软件的基础》一书中定义，迭代器用于从集合中取出元素，而生成器用于生成元素。在Python社区中，大多数都把迭代器和生成器视作同一个概念。

> ![](https://bugs.python.org/file47357/base.png)
> 生成器（Generator）继承于迭代器（Iterator），因此，所有的生成器都是迭代器。

In [1]:
import collections

In [2]:
collections.abc.Generator.__mro__

(collections.abc.Generator,
 collections.abc.Iterator,
 collections.abc.Iterable,
 object)

In [8]:
# help(collections.abc.Iterable)
# help(collections.abc.Iterator)
# help(collections.abc.Generator)

在Python中，所有集合都可以叠戴。在Python语言内部，迭代器用于支持：

* for循环
* 构建和扩展集合类型
* 逐行遍历文本文件
* 列表推导、字典推导和集合推导
* 元祖拆包
* 调用函数时，使用*拆包参数

## 可迭代对象与迭代器的对比

```
可迭代对象

使用iter内置函数可以获取迭代器对象。如果对象实现了__iter__方法，那么对象是可迭代。序列都可以迭代；实现了__getitem__方法，而且其参数时从开始的索引，这种对象也可以迭代。
```

标准的迭代器接口有两个方法：

```
__next__

返回下一个可用元素，如果没有元素了，抛出StopIteration异常。

__iter__

返回self，以便在应该使用可迭代对象的地方使用迭代器，例如在for循环中。
```

In [9]:
import collections

In [10]:
help(collections.abc.Iterator)

Help on class Iterator in module collections.abc:

class Iterator(Iterable)
 |  Method resolution order:
 |      Iterator
 |      Iterable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __iter__(self)
 |  
 |  __next__(self)
 |      Return the next item from the iterator. When exhausted, raise StopIteration
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  __subclasshook__(C) from abc.ABCMeta
 |      Abstract classes can override this to customize issubclass().
 |      
 |      This is invoked early on by abc.ABCMeta.__subclasscheck__().
 |      It should return True, False or NotImplemented.  If it returns
 |      NotImplemented, the normal algorithm is used.  Otherwise, it
 |      overrides the normal algorithm (and the outcome is cached).
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = froze

Reference: https://github.com/python/cpython/blob/master/Lib/_collections_abc.py

```python
class Iterable(metaclass=ABCMeta):

    __slots__ = ()

    @abstractmethod
    def __iter__(self):
        while False:
            yield None

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterable:
            return _check_methods(C, "__iter__")
        return NotImplemented


class Iterator(Iterable):

    __slots__ = ()

    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration'
        raise StopIteration

    def __iter__(self):
        return self

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            return _check_methods(C, '__iter__', '__next__')
        return NotImplemented
```

```
Iterators in Python aren't a matter of type but of protocol. A large and changing number of builtin types implement *some* flavor of iterator. Don't check the type! Use hasattr to check for both "__iter__" and "__next__" attributes instead.
```

```
迭代器

迭代器是这样的对象：实现了无参数的__next__方法，返回序列中的下一个元素；如果没有元素了，那么抛出StopIteration异常。Python中的迭代器还实现了__iter__方法，因此迭代器也可以迭代。
```

## Sentence类

In [14]:
import re
import reprlib

In [15]:
RE_WORD = re.compile('\w+')

In [16]:
class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [18]:
s = Sentence('"The time has come", the Walrus said.')

In [19]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [29]:
print(s)

Sentence('"The time ha... Walrus said.')


In [30]:
len(s)

7

### 序列可以迭代的原因：iter函数

解释器需要迭代对象x时，会自动调用iter(x)。

内置的iter函数有一下作用：

```
1. 检查对象是否实现了__iter__方法，如果实现了就调用它，获取一个迭代器。
2. 如果没有实现__iter__方法，但是实现了__getitem__方法，，Python会创建一个迭代器，尝试按顺序（从索引0开始）获取元素。
3. 如果尝试失败，Python抛出TypeError异常，通常会提示"C object is not iterable"，其中C是目标对象所属的类。
```

```
任何Python序列都可以迭代的原因是，它们都实现了__getitem__方法。其实，标准的序列也都实现了__iter__方法。
```

## Sentence类：典型的迭代器

In [31]:
import re
import reprlib

In [32]:
RE_WORD = re.compile('\w+')

In [56]:
class SentenceIterator:

    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
            
        self.index += 1
        return word

In [57]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
        
    def __iter__(self):
        return SentenceIterator(self.words)

In [58]:
it = iter(Sentence('"The time has come", the Walrus said.'))

In [59]:
it

In [60]:
list(it)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

### 把Sentence变成迭代器：坏主意

```
构建可迭代的对象和迭代器时经常会出现错误，原因是混淆了了二者。可迭代的对象有个__iter__方法，每次都实例化一个新的迭代器；而迭代器要实现__next__方法，返回单个单元，此外还要实现__iter__方法，返回迭代器本身。
```

因此，迭代器可以迭代，但是可迭代的对象不是迭代器。

```
除了__iter__方法之外，你可能还想在Sentence类中实现__next__方法，让Sentence实例既是可迭代的对象，也是自身的迭代器。可是，这种想法非常糟糕。根据有大量Python代码审查经验的Alex Matelli所说，这也是常见的反模式。
```

《设计模式》一书讲解迭代器设计模式时，在“适用性”一节中说：

迭代器模式可用来:

* 访问一个聚合对象的内容无需暴露它的内部表示
* 支持对聚合对象多种遍历
* 为遍历不同的聚合结构提供一个统一的借口

为了“支持多种遍历”，必须能从同一个可迭代的实例中获取多个独立迭代器，而且各个迭代器要维护自身的内部状态，因此这一模式正确的实现方式是，每次调用iter(my_iterable)都新建一个独立的迭代器。

> **可迭代的对象一定不能是自身的迭代器。也就是说，可迭代的对象必须实现`__iter__`方法，但不能实现`__next__`方法。**

## Sentence类：生成器函数

In [61]:
import re
import reprlib

In [62]:
RE_WORD = re.compile('\w+')

In [122]:
class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word
            
        return # 会触发生成器抛出StopIteration异常


In [123]:
s = Sentence('"The time has come", the Walrus said.')

In [124]:
iter(s)

<generator object Sentence.__iter__ at 0x10ad7b7d0>

In [125]:
list(iter(s))

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

### 生成器函数的工作原理

只要Python函数的定义中有`yield`关键字，该函数就是生成器函数。调用生成器函数时，会返回一个生成器对象。也就是说，生成器函数是生成器工厂。

> 普通的函数与生成器函数在句法上唯一的区别是，在后者定义中有yield关键字。

In [87]:
def gen_123():
    yield 1
    yield 2
    yield 3

In [88]:
g = gen_123()

In [89]:
print(next(g))
print(next(g))
print(next(g))

1
2
3


生成器函数会创建一个生成器对象，包装生成器函数的定义体。把生成器传给`next(...)`函数时，生成器函数会向前，执行函数定义体中的下一个yield语句，返回产生的值，并在函数定义体的当前位置暂停。最终，函数的定义体返回时，外层的生成器对象会抛出StopIteration异常--这一点与迭代器协议一致。

> **调用生成器函数返回生成器，生成器产出或产生值。生成器不会以常规的方式“返回”值：生成器函数定义体中的return语句会触发生成器抛出StopIteration异常。**

In [91]:
help(collections.abc.Generator)

Help on class Generator in module collections.abc:

class Generator(Iterator)
 |  Method resolution order:
 |      Generator
 |      Iterator
 |      Iterable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __next__(self)
 |      Return the next item from the generator.
 |      When exhausted, raise StopIteration.
 |  
 |  close(self)
 |      Raise GeneratorExit inside generator.
 |  
 |  send(self, value)
 |      Send a value into the generator.
 |      Return next yielded value or raise StopIteration.
 |  
 |  throw(self, typ, val=None, tb=None)
 |      Raise an exception in the generator.
 |      Return next yielded value or raise StopIteration.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  __subclasshook__(C) from abc.ABCMeta
 |      Abstract classes can override this to customize issubclass().
 |      
 |      This is invoked early on by abc.ABCMeta.__subclasscheck__().
 |      It should ret

## Sentence类：惰性实现

设置Iterator接口时考虑到了惰性：next(my_iterator)一次生成一个元素。惰性的反义词是急迫，其实，惰性求值（lazy evaluation）和及早求值（eager evaluation）是编程理论方面的技术数据。

```
The Iterator interface is designed to be lazy: next(my_iterator) produces one item at a time. The opposite of lazy is eager: lazy evaluation and eager evaluation are actual technical terms in programming languagure theory.
```

re.finditer函数是re.findall函数的惰性版本，返回的不是列表，而是一个生成器，按需生成re.MatchObject实例。如果有很多匹配，re.finditer函数能节省大量内存。

In [92]:
import re
import reprlib

In [93]:
RE_WORD = re.compile('\w+')

In [137]:
class Sentence:
    
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        print('lazy evaluation')
        
        for match in RE_WORD.finditer(self.text):
            yield match.group()

In [140]:
iter(Sentence(''))

<generator object Sentence.__iter__ at 0x10ad7b150>

In [138]:
list(Sentence('"The time has come", the Walrus said.'))

lazy evaluation


['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

## Sentence类：生成器表达式

生成器表达式可以理解为列表推导的惰性版本：不会迫切地构建列表，而是返回一个生成器，按需惰性生成元素。也就是说，如果列表推到是制造列表的工厂，那么生成器表达式就是制造生成器的工厂。

```
A generator expression can be understood as a lazy version of a list comprehension: it does not eagerly build a list, but returns a generator that will lazily produce the items on demand. In other words, if a list comprehension is factory of lists, a generator expression is a factory of generators.
```

In [141]:
import re
import reprlib

In [142]:
RE_WORD = re.compile('\w+')

In [145]:
class Sentence:

    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        print('Generator expession.')
        return (match.group() for match in RE_WORD.finditer(self.text))

In [146]:
list(Sentence('Machine Learing'))

Generator expession.


['Machine', 'Learing']

In [147]:
iter(Sentence('Machine Learning'))

Generator expession.


<generator object Sentence.__iter__.<locals>.<genexpr> at 0x10ad7b350>

这是不是生成器函数了（没有yield），而是使用生成器表达式构建生成器，然后将其返回。

> [Python Reference generator expressions](https://docs.python.org/3/reference/expressions.html#generator-expressions)
> generator expression yields a new generator object.

In [154]:
list(('%s * %s = %s' % (x, y, x*y) for x in range(3) for y in range(x, x+3)))

['0 * 0 = 0',
 '0 * 1 = 0',
 '0 * 2 = 0',
 '1 * 1 = 1',
 '1 * 2 = 2',
 '1 * 3 = 3',
 '2 * 2 = 4',
 '2 * 3 = 6',
 '2 * 4 = 8']

## 何时使用生成器表达式

生成器表达式是创建生成器的简洁句法，这样无需定义函数再调用。不过，生成器函数灵活的多，可以使用多个语句实现复杂的逻辑，也可以作为协程使用。

遇到简单的情况时，可以生成器表达式，因为这样扫一眼就只代码的作用。

选择使用那种句法很容判断：如果生成器表达式要分多行些，我倾向于定义生成器函数，以便提高可读性。

## 等差数列生成器

In [157]:
class ArithmeticProgression:
    
    def __init__(self, start, end, step=1):
        self.start = start
        self.end = end
        self.step = step
        
    def __iter__(self):
        index = 0
        result = self.start

        while result < self.end:
            yield result
            index += 1
            result = self.start + self.step * index

In [159]:
list(ArithmeticProgression(1, 10, 2))

[1, 3, 5, 7, 9]

## 标准库的生成器函数

[Python Reference: itertools - Functions create iterators for efficient looping](https://docs.python.org/3/library/itertools.html)

### 用于过滤的生成器函数

|模块|函数|说明|
|---|---|----|
|itrtools|compress(it, selector_it)||
|ittootls|dropwhile(predicate, it) ||
|built-in|filter(predicate, it)||
|itertools|filterfalse(predicate, it)||
|itertools|islice(it, stop) or islice(it, start, stop, step=1)||
|itertools|takewhile(predicate, it)||

In [209]:
import itertools

In [213]:
def vowel(c):
    return c.lower() in 'aeiou'

In [214]:
list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [215]:
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [216]:
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [249]:
list(itertools.compress('Aardvark', (1,0,1,1,0,1,)))

['A', 'r', 'd', 'a']

### 用于映射的生成器函数

|模块|函数|说明|
|---|----|---|
|itertools|accumulate(it, [func])||
|built-in|enumerate(itrable, start=0)||
|built-in|map(func, it1, [it2, ..., itN])||
|itertools|startmap(func, it)||

In [218]:
import random

In [229]:
sample = [random.randint(1, 9) for num in range(10)]
sample

[7, 4, 2, 2, 6, 6, 3, 6, 9, 6]

In [230]:
list(itertools.accumulate(sample))

[7, 11, 13, 15, 21, 27, 30, 36, 45, 51]

In [231]:
list(itertools.accumulate(sample, min))

[7, 4, 2, 2, 2, 2, 2, 2, 2, 2]

In [232]:
list(itertools.accumulate(sample, max))

[7, 7, 7, 7, 7, 7, 7, 7, 9, 9]

In [235]:
list(enumerate(sample, 1))

[(1, 7),
 (2, 4),
 (3, 2),
 (4, 2),
 (5, 6),
 (6, 6),
 (7, 3),
 (8, 6),
 (9, 9),
 (10, 6)]

In [236]:
import operator

In [239]:
list(itertools.starmap(operator.mul, enumerate('abcde', 1)))

['a', 'bb', 'ccc', 'dddd', 'eeeee']

### 合并多个可迭代对象的生成器函数

|模块|函数|说明|
|---|----|---|
|itertools|chain(it1, ..., itN)||
|itertools|chain.from_iterable(it)||
|itertools|product(it1, ..., itN, repeat=1)|计算笛卡尔积|
|built-in|zip(it1, ..., itN)||
|itertools|zip_longest(it1, ..., itN, fillvalue=None)||

In [241]:
list(itertools.chain('ABCD', range(2)))

['A', 'B', 'C', 'D', 0, 1]

In [242]:
list(itertools.chain.from_iterable(enumerate('ABCD')))

[0, 'A', 1, 'B', 2, 'C', 3, 'D']

In [243]:
list(zip('ABC', range(3)))

[('A', 0), ('B', 1), ('C', 2)]

In [244]:
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

In [247]:
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [248]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

### 把输入的各个元素扩展成多个暑促元素的生成器函数

|模块|函数|说明|
|---|----|---|
|itertools|combiantions(it, out_len)||
|itertools|combination_with_replacement(it, out_len)||
|itertools|count(start=0, step=1)||
|itertools|cycle(it)||
|itertools|permutations(it, out_len=None)||
|itertools|repeat(item, [times])||

In [250]:
cy = itertools.cycle('ABC')

In [251]:
next(cy)

'A'

In [252]:
list(itertools.islice(cy, 7))

['B', 'C', 'A', 'B', 'C', 'A', 'B']

In [254]:
list(itertools.repeat(7,3))

[7, 7, 7]

In [263]:
list(map(operator.mul, range(11), itertools.repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [256]:
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [257]:
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [264]:
list(itertools.permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

### 用于重新排列元素的生成器

|模块|函数|说明|
|---|----|---|
|itertools|gropby(it, key=None)||
|built-in|reversed(seq)||
|itertools|tee(it, n=2)||

In [265]:
list(itertools.groupby('AAAABBCCC'))

[('A', <itertools._grouper at 0x10c24e490>),
 ('B', <itertools._grouper at 0x10c3d6cd0>),
 ('C', <itertools._grouper at 0x10c3d6590>)]

In [267]:
for char, group in itertools.groupby('AAAABBCCC'):
    print(char, '->', list(group))

A -> ['A', 'A', 'A', 'A']
B -> ['B', 'B']
C -> ['C', 'C', 'C']


In [269]:
list(itertools.tee('ABC', 2))

[<itertools._tee at 0x10c09abe0>, <itertools._tee at 0x10c09a4b0>]

In [270]:
g1, g2 = itertools.tee('ABC', 2)

In [271]:
print(next(g1))
print(next(g2))
print(list(g1))

A
A
['B', 'C']


In [274]:
list(zip(*itertools.tee('ABC', 2)))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

## 可迭代的归约函数

函数接受一个可迭代的对象，然后返回单个结果。这个函数就叫“归约”函数。

```
* all
* any
* max
* min
* sum
* functools.reduce
```

## 深入分析iter函数

在Python中迭代对象x时，会调用iter(x)。

iter函数还有一个鲜为人知的用法：传入两个参数，使用常规的函数或可调用的对象创建迭代器。第一个参数必须是可调用的对象，用于不断调用，产出各个值，第二个参数是哨兵，当可调用对象返回这个值时，触发迭代器抛出StopIteration异常，而不产出哨兵。

In [161]:
help(iter)

Help on built-in function iter in module builtins:

iter(...)
    iter(iterable) -> iterator
    iter(callable, sentinel) -> iterator
    
    Get an iterator from an object.  In the first form, the argument must
    supply its own iterator, or be a sequence.
    In the second form, the callable is called until it returns the sentinel.



In [162]:
import random

In [164]:
def d6():
    return random.randint(1, 7)

In [187]:
for i in iter(d6, 2):
    print(i)

1
4
3


## 把生成器当成协程

Python2.2引入了yield关键字实现的生成器，Python2.5实现了[PEP342-- Coroutine via Enhenced Generators](https://www.python.org/dev/peps/pep-0342/)。这个提案为生成器对象添加了额外的方法和功能，其中最值得关注的是.send()方法。

In [190]:
import collections
help(collections.abc.Generator)

Help on class Generator in module collections.abc:

class Generator(Iterator)
 |  Method resolution order:
 |      Generator
 |      Iterator
 |      Iterable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __next__(self)
 |      Return the next item from the generator.
 |      When exhausted, raise StopIteration.
 |  
 |  close(self)
 |      Raise GeneratorExit inside generator.
 |  
 |  send(self, value)
 |      Send a value into the generator.
 |      Return next yielded value or raise StopIteration.
 |  
 |  throw(self, typ, val=None, tb=None)
 |      Raise an exception in the generator.
 |      Return next yielded value or raise StopIteration.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  __subclasshook__(C) from abc.ABCMeta
 |      Abstract classes can override this to customize issubclass().
 |      
 |      This is invoked early on by abc.ABCMeta.__subclasscheck__().
 |      It should ret

```
与__next__()方法一样，.send()方法也可以使生成器前进到下一个yield语句。不过，send()方法允许生成器把客户端的数据发给自己。也就是说，send()方法允许客户端和生成器之间双向交换数据。而__next__()方法只允许客户从生成器中获取数据。
```

这一项重要的“改进”，甚至改变了生成器的本性：生成器就变身协程了。

* 生成器用于供迭代到数据
* 协程是数据的消费者
* 协程于迭代无关
* 注意，虽然协程中会使用yield产出值，但这与迭代无关

## 延伸阅读

**生成器与迭代器的语义对比**

思考迭代器于生成器之间的关系时，至少可以从三方面入手。

第一方面是接口。

```
Python的迭代器协议定义了两个方法：__next__和__iter__。生成器对象实现了这两个方法，因此从这方面来看，所有生成器都是迭代器。由此可以得知，内置的`enumerate()`函数创建的对象是迭代器。
```

In [201]:
import collections

e = enumerate('ABC')
isinstance(e, collections.abc.Iterator)

True

In [199]:
# help(e)

第二方面是实现方式。

Python生成器有两种实现方式：含有yield关键字的函数，或者生成器表达式。调用生成器函数或者生成器表达式得到的生成器对象属于语言内部`GeneratorType`类型。

[Python Reference GeneratorType](https://docs.python.org/3/library/types.html#types.GeneratorType)

```
GeneratorType
The type of generator-iterator objects, created by generator functions.
```

第三个方面是概念。

根据《设计模式》一书的定义，在典型的迭代器设计模式中，迭代器用于遍历集合，从中产出元素。而生成器无需遍历集合就能生成值，例如range()函数。

## Reference

* [Python Refrence itertools -- Functions creating iterators for efficient loopings](https://docs.python.org/3/library/itertools.html)
* [Python PEP 342 -- Coroutines via Enhanced Generators](https://www.python.org/dev/peps/pep-0342/)
* [Python Reference generator expressions](https://docs.python.org/3/reference/expressions.html#generator-expressions)
* [Python Reference collections.abc -- Abstract Base Classes for Containers](https://docs.python.org/3/library/collections.abc.html#module-collections.abc)
* *Fuent Python Chapter 14*